In [1]:
import matplotlib.pyplot as pyplot
%matplotlib inline

In [2]:
# loading scripts are in scripts/dataset.py
from scripts import dataset
import chainer
import os

# example code from tests/test_snapshot.py
dataset_name = 'lsp'
#dataset_name = 'lsp'

if dataset_name == 'mpii':
    DATASET_ROOT = os.path.expanduser('~/data/mpii')  # MPII dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(MPII_DATASET_ROOT, 'images')
    symmetric_joints = "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -100500
else:
    DATASET_ROOT = os.path.expanduser('~/data/lsp')  # LSP dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(DATASET_ROOT, 'images')
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1

test_dataset = dataset.PoseDataset(
        TEST_CV_FILEPATH,
        IMG_PATH_PREFIX, 227,
        fliplr=False, rotate=False,
        shift=None,
        bbox_extension_range=(1.0, 1.0),
        coord_normalize=True,
        gcn=True,
        fname_index=0,
        joint_index=1,
        symmetric_joints=symmetric_joints,
        ignore_label=ignore_label,
        should_return_bbox=True,
        should_downscale_images=True,
        downscale_height=400
)

209it [00:00, 2085.51it/s]

Reading dataset from /home/yyoo/data/lsp/test_joints.csv
Downscale images to the height 400px


1000it [00:00, 2074.33it/s]

Joints shape: (14, 2)


In [3]:
whos

Variable           Type           Data/Info
-------------------------------------------
DATASET_ROOT       str            /home/yyoo/data/lsp
IMG_PATH_PREFIX    str            /home/yyoo/data/lsp/images
TEST_CV_FILEPATH   str            /home/yyoo/data/lsp/test_joints.csv
chainer            module         <module 'chainer' from '/<...>ges/chainer/__init__.py'>
dataset            module         <module 'scripts.dataset'<...>pose/scripts/dataset.py'>
dataset_name       str            lsp
ignore_label       int            -1
os                 module         <module 'os' from '/usr/lib/python3.5/os.py'>
pyplot             module         <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
symmetric_joints   str            [[8, 9], [7, 10], [6, 11]<...>, [2, 3], [1, 4], [0, 5]]
test_dataset       PoseDataset    <scripts.dataset.PoseData<...>object at 0x7fdcc90a1358>


In [4]:
test_dataset?

In [5]:
test_dataset??

In [7]:
from chainer import iterators
test_iterator = iterators.MultiprocessIterator(
        test_dataset, batch_size=128,
        repeat=False, shuffle=False,
        n_processes=1, n_prefetch=1)

In [18]:
##code from def test_net(test_dataset, test_iterator, dataset_name, snapshot_path):
from scripts import regressionnet

## set the checkpoint that you want to use
snapshot_path=os.path.expanduser('~/src/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-150000')

if dataset_name not in ['lsp', 'mpii']:
    raise ValueError('Unknown dataset: {}'.format(dataset_name))

net, loss_op, pose_loss_op, train_op = regressionnet.create_regression_net( # in scripts.
        n_joints=16 if dataset_name == 'mpii' else 14,
        init_snapshot_path=snapshot_path,
        is_resume=True,
        net_type='Alexnet',
        optimizer_type='momentum',
        gpu_memory_fraction=0.32)  # Set how much GPU memory to reserve for the network
print(snapshot_path)

Alexnet::__create_architecture()
Initializing conv1 with random
Initializing conv2 with random
Initializing conv3 with random
Initializing conv4 with random
Initializing conv5 with random
Initializing fc6 with random
Initializing fc7 with random
Initializing fc8 with random
Initializing layer 99 with random
Restoring everything from snapshot and resuming
INFO:tensorflow:Restoring parameters from /home/yyoo/src/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-150000


INFO:tensorflow:Restoring parameters from /home/yyoo/src/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-150000


Creating optimizer momentum
Conv LR: Tensor("lr/conv_lr:0", shape=(), dtype=float32), FC LR: Tensor("lr/fc_lr:0", shape=(), dtype=float32)
INFO:tensorflow:Summary name grad_norms/conv1/weight:0 is illegal; using grad_norms/conv1/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/conv1/weight:0 is illegal; using grad_norms/conv1/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/conv1/bias:0 is illegal; using grad_norms/conv1/bias_0 instead.


...

INFO:tensorflow:Summary name grad_norms/fc_regression/weight:0 is illegal; using grad_norms/fc_regression/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/weight:0 is illegal; using grad_norms/fc_regression/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/bias:0 is illegal; using grad_norms/fc_regression/bias_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/bias:0 is illegal; using grad_norms/fc_regression/bias_0 instead.


uninit vars: ['conv1/weight/Momentum:0', 'conv1/bias/Momentum:0', 'conv2/weight/Momentum:0', 'conv2/bias/Momentum:0', 'conv3/weight/Momentum:0', 'conv3/bias/Momentum:0', 'conv4/weight/Momentum:0', 'conv4/bias/Momentum:0', 'conv5/weight/Momentum:0', 'conv5/bias/Momentum:0', 'fc6/weight/Momentum:0', 'fc6/bias/Momentum:0', 'fc7/weight/Momentum:0', 'fc7/bias/Momentum:0', 'fc_regression/weight/Momentum:0', 'fc_regression/bias/Momentum:0']
Elapsed time for finding uninitialized variables: 0.77s
Elapsed time to init them: 0.22s
/home/yyoo/src/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-150000


In [19]:
import numpy as np

for ext in np.linspace(1.0, 2.0, 6, True):
    print('\n====================')
    print('BBOX EXTENSION:', ext)
    test_dataset.bbox_extension_range = (ext, ext)
    regressionnet.evaluate_pcp(net, pose_loss_op, test_iterator, None, # in scripts.
                               dataset_name=dataset_name,
                               tag_prefix='test')

  0%|          | 0/8 [00:00<?, ?it/s]


BBOX EXTENSION: 1.0
1000


100%|██████████| 8/8 [00:04<00:00,  1.85it/s]


Step 150000 test/pose_loss = 0.078
lsp


/home/yyoo/src/deeppose/poseevaluation/pcp.py:88: RuntimeWarning: divide by zero encountered in float_scalars
  gt_joints[i]['sticks'][stick_id, :2]) / gt_stick_len
/home/yyoo/src/deeppose/poseevaluation/pcp.py:90: RuntimeWarning: divide by zero encountered in float_scalars
  gt_joints[i]['sticks'][stick_id, 2:]) / gt_stick_len


...

Step 150000 test/pose_loss = 0.027
lsp
Step	 150000	 test/mPCP	 0.454
Step 150000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.360	0.856	0.318	0.134	0.624	0.433	0.454
lsp
Step	 150000	 test/mRelaxedPCP	 0.553
Step 150000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.468	0.905	0.421	0.221	0.740	0.564	0.553
lsp


  0%|          | 0/8 [00:00<?, ?it/s]

Step	 150000	 test/mPCKh	 0.367
Step	 150000	 test/mSymmetricPCKh	 0.380
Step 150000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.380	0.559	0.432	0.328	0.193	0.583	0.394	0.171

BBOX EXTENSION: 2.0
1000


100%|██████████| 8/8 [00:04<00:00,  1.86it/s]


Step 150000 test/pose_loss = 0.027
lsp
Step	 150000	 test/mPCP	 0.452
Step 150000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.354	0.858	0.313	0.132	0.623	0.431	0.452
lsp
Step	 150000	 test/mRelaxedPCP	 0.552
Step 150000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.464	0.905	0.420	0.220	0.740	0.564	0.552
lsp
Step	 150000	 test/mPCKh	 0.366
Step	 150000	 test/mSymmetricPCKh	 0.378
Step 150000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.373	0.559	0.432	0.322	0.193	0.582	0.394	0.169
